In [2]:
import requests, urllib2, gzip, urllib
import pandas as pd
import numpy as np
import math

In [3]:
url = 'http://s3.amazonaws.com/thedataincubator/coursedata/mldata/train.txt.gz'

In [4]:
#Download file
urllib.urlretrieve(url,'Weather')

weather_list = []
with gzip.open('Weather', 'rb') as f:
    for line in f.readlines():
        info_dict = {}
        info = line.split()
        info_dict['month_sin'] = math.sin(2*math.pi*(float(info[1])/12))
        info_dict['month_cos'] = math.cos(2*math.pi*(float(info[1])/12))
        info_dict['hour_sin'] = math.sin(2*math.pi*(float(info[3])/24))
        info_dict['hour_cos'] = math.cos(2*math.pi*(float(info[3])/24))
        info_dict['temp'] = int(info[4])
        info_dict['city'] = info[12]
        if info_dict['temp'] != -9999.0:
            weather_list.append(info_dict)
        
print weather_list[0:2]
print len(weather_list)


[{'city': 'bos', 'hour_sin': 0.0, 'temp': -11, 'month_sin': 0.49999999999999994, 'month_cos': 0.8660254037844387, 'hour_cos': 1.0}, {'city': 'bos', 'hour_sin': 0.25881904510252074, 'temp': -6, 'month_sin': 0.49999999999999994, 'month_cos': 0.8660254037844387, 'hour_cos': 0.9659258262890683}]
515581


In [5]:
weather_df = pd.DataFrame(weather_list)
weather_df.head(10)
#weather_df.describe()

,city,hour_cos,hour_sin,month_cos,month_sin,temp
0,bos,1.000000e+00,0.000000,0.866025,0.5,-11
1,bos,9.659258e-01,0.258819,0.866025,0.5,-6
2,bos,8.660254e-01,0.500000,0.866025,0.5,-17
3,bos,7.071068e-01,0.707107,0.866025,0.5,-17
4,bos,5.000000e-01,0.866025,0.866025,0.5,-17
5,bos,2.588190e-01,0.965926,0.866025,0.5,-22
6,bos,6.123234e-17,1.000000,0.866025,0.5,-28
7,bos,-2.588190e-01,0.965926,0.866025,0.5,0
8,bos,-5.000000e-01,0.866025,0.866025,0.5,-11
9,bos,-7.071068e-01,0.707107,0.866025,0.5,-28


In [6]:
four_list = weather_df.values.tolist()
print four_list[:2]

[['bos', 1.0, 0.0, 0.8660254037844387, 0.49999999999999994, -11], ['bos', 0.9659258262890683, 0.25881904510252074, 0.8660254037844387, 0.49999999999999994, -6]]


In [7]:
#Check for different cities.
check_df = weather_df.groupby('city').count()
check_df.head(10)

,hour_cos,hour_sin,month_cos,month_sin,temp
city,,,,,
bal,102623,102623,102623,102623,102623
bos,102598,102598,102598,102598,102598
chi,105126,105126,105126,105126,105126
nyc,102622,102622,102622,102622,102622
phi,102612,102612,102612,102612,102612


In [8]:
#Create a dataframe of values for each city after first creating a list of entries for each city.
bos_list = []
bal_list = []
chi_list = []
nyc_list = []
phi_list = []

for item in four_list:
    if item[0] == 'bos':
        bos_list.append(item)
    if item[0] == 'bal':
        bal_list.append(item)
    if item[0] == 'chi':
        chi_list.append(item)
    if item[0] == 'nyc':
        nyc_list.append(item)
    if item[0] == 'phi':
        phi_list.append(item)

columns = ['city', 'hr_cos', 'hr_sin', 'mth_cos', 'mth_sin', 'temp']        
        
bos_df = pd.DataFrame(bos_list, columns = columns)
bal_df = pd.DataFrame(bal_list, columns = columns)
chi_df = pd.DataFrame(chi_list, columns = columns)
nyc_df = pd.DataFrame(nyc_list, columns = columns)
phi_df = pd.DataFrame(phi_list, columns = columns)



print bos_df.head()
        
    

  city    hr_cos    hr_sin   mth_cos  mth_sin  temp
0  bos  1.000000  0.000000  0.866025      0.5   -11
1  bos  0.965926  0.258819  0.866025      0.5    -6
2  bos  0.866025  0.500000  0.866025      0.5   -17
3  bos  0.707107  0.707107  0.866025      0.5   -17
4  bos  0.500000  0.866025  0.866025      0.5   -17


In [9]:
#Create numpy arrays of all the target and data points for each city's model.
data_points = ['hr_cos', 'hr_sin', 'mth_cos', 'mth_sin']

target_bos = np.array(bos_df['temp'])
data_bos = np.array(bos_df[data_points])

target_bal = np.array(bal_df['temp'])
data_bal = np.array(bal_df[data_points])

target_chi = np.array(chi_df['temp'])
data_chi = np.array(chi_df[data_points])

target_nyc = np.array(nyc_df['temp'])
data_nyc = np.array(nyc_df[data_points])

target_phi = np.array(phi_df['temp'])
data_phi = np.array(phi_df[data_points])




In [10]:
#Fit linear model for each city.
from sklearn import linear_model

bos_clf = linear_model.Ridge(alpha=0.5)
bos_clf = bos_clf.fit(data_bos, target_bos)

bal_clf = linear_model.Ridge(alpha=0.5)
bal_clf = bal_clf.fit(data_bal, target_bal)

chi_clf = linear_model.Ridge(alpha=0.5)
chi_clf = chi_clf.fit(data_chi, target_chi)

nyc_clf = linear_model.Ridge(alpha=0.5)
nyc_clf = nyc_clf.fit(data_nyc, target_nyc)

phi_clf = linear_model.Ridge(alpha=0.5)
phi_clf = phi_clf.fit(data_phi, target_phi)


In [15]:
#Test models.
test = "2000 01 01 02   xx   -78 10211   230    36     0     0 -9999 bos"

info = test.split()

check = [math.sin(2*math.pi*(float(info[3])/24)), math.cos(2*math.pi*(float(info[3])/24)), math.sin(2*math.pi*(float(info[1])/12)),math.cos(2*math.pi*(float(info[1])/12)) ]

model = bos_clf
prediction = model.predict(check)

print prediction

[-16.90110661]


In [13]:
#Store the instances using dill.
import dill
with open('bos_est_file','w') as f:
    dill.dump(bos_clf,f)
    
import dill
with open('bal_est_file','w') as f:
    dill.dump(bal_clf,f)
    
import dill
with open('chi_est_file','w') as f:
    dill.dump(chi_clf,f)
    
import dill
with open('nyc_est_file','w') as f:
    dill.dump(nyc_clf,f)
    
import dill
with open('phi_est_file','w') as f:
    dill.dump(phi_clf,f)
    